In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import mpld3
#mpld3.enable_notebook()

In [ ]:
def normalise(means, errors, baseline, keep=False):
    new_means = {}
    new_errors = {}
    num_tests = len(means[baseline])
    for version in means:
        if version == baseline and not keep:
            continue
        new_means[version] = [means[version][i]/means[baseline][i] for i in range(num_tests)]
        if errors is not None:
            new_errors[version] =[errors[version][i]/means[baseline][i] for i in range(num_tests)]
    return new_means, new_errors

In [ ]:
def bar_plot(ax, data, errors=None, colors=None, total_width=0.8, single_width=1, legend=False, capsize=3):
    """Draws a bar plot with multiple bars per data point.

    Parameters
    ----------
    ax : matplotlib.pyplot.axis
        The axis we want to draw our plot on.

    data: dictionary
        A dictionary containing the data we want to plot. Keys are the names of the
        data, the items is a list of the values.

        Example:
        data = {
            "x":[1,2,3],
            "y":[1,2,3],
            "z":[1,2,3],
        }

    errors: dictionary, optional
        Dictionary of standard deviations, corresponding structure to data

    colors : array-like, optional
        A list of colors which are used for the bars. If None, the colors
        will be the standard matplotlib color cyle. (default: None)

    total_width : float, optional, default: 0.8
        The width of a bar group. 0.8 means that 80% of the x-axis is covered
        by bars and 20% will be spaces between the bars.

    single_width: float, optional, default: 1
        The relative width of a single bar within a group. 1 means the bars
        will touch eachother within a group, values less than 1 will make
        these bars thinner.

    legend: bool, optional, default: True
        If this is set to true, a legend will be added to the axis.
    """

    # Check if colors where provided, otherwhise use the default color cycle
    if colors is None:
        colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

    # Number of bars per group
    n_bars = len(data)

    # The width of a single bar
    bar_width = total_width / n_bars

    # List containing handles for the drawn bars, used for the legend
    bars = []

    # Iterate over all data
    for i, (name, values) in enumerate(data.items()):
        # The offset in x direction of that bar
        x_offset = (i - n_bars / 2) * bar_width + bar_width / 2

        # Draw a bar for every value of that type
        for x, y in enumerate(values):
            if errors is None:
                bar = ax.bar(x + x_offset, y, width=bar_width * single_width, color=colors[i % len(colors)])
            else:
                err = errors[name][x]
                bar = ax.bar(x + x_offset, y, yerr=err, error_kw=dict(capsize=capsize),
                             width=bar_width * single_width, color=colors[i % len(colors)])

        # Add a handle to the last drawn bar, which we'll need for the legend
        bars.append(bar[0])

#    # Draw legend if we need
#    if legend:
#        ax.legend(bars, data.keys())
    # return the handlers/labels for a legend
    if legend:
        return bars, data.keys()

## Comparison of All optimisations, just IR, just Wasm, None
All done with optimised pattern matching, no GC, and 3 iterations where optimisation passes are being applied.

In [ ]:
data = {}
versions = ["None", "Wasm", "IR",  "All"]
# Check file to see formatting
with open("../../wasm-of-ocaml/benchmarks/evaluation/optimisations.txt") as f:
    f.readline()
    for version in versions:
        f.readline()
        line = f.readline().strip().split()
        while line != []:
            if line[0] not in data:
                data[line[0]] = {}
            data[line[0]][version] = \
              {"time" : float(line[1]), "error" : float(line[2]), "heap": float(line[3]), "filesize" : float(line[4])}
            line = f.readline().strip().split()
tests = data.keys()

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(10,10))
ax = axs[0]
times = {version: [data[test][version]["time"] for test in tests] for version in versions}
errors = {version: [data[test][version]["error"] for test in tests] for version in versions}
times, errors = normalise(times, errors, "None", True)
#times = {lang : [means[lang][b] for b in benchmarks if b != "arith"] for lang in means if lang != "Grain"}
bar_plot(ax, times, errors=errors)
ax.set_title("Execution time")
ax.set_xticks([])
ax.set_ylabel("relative time")

ax = axs[1]
heap = {version: [data[test][version]["heap"] for test in tests] for version in versions}
heap, _ = normalise(heap, None, "None", True)
handlers, labels = bar_plot(ax, heap, legend=True)
ax.set_title("Heap usage")
ax.set_xticks([])

ax = axs[2]
sizes = {version: [data[test][version]["filesize"] for test in tests] for version in versions}
sizes, _ = normalise(sizes, None, "None", True)
handlers, labels = bar_plot(ax, sizes, legend=True)
ax.set_title("Filesize")
ax.set_xticks(range(len(tests)))
ax.set_xticklabels(tests, rotation=90)
ax.set_ylabel("size (bytes)");

fig.legend(handlers, labels, loc='center left');
plt.subplots_adjust(left=0.15)

Optimisations at both the WebAssembly and IR level reduce filesize across the board. TODO: Average percentage/range?  
In many cases, execution time is significantly reduced by the IR level optimisations, although the WebAssembly optimisations have little impact on performance, possibly making it slightly worse in some cases but within 1 standard deviation of the original execution time. The are a couple of tests where execution time is not improved, `nbody` is an imperative style program performing floating point calculations.

In [ ]:
ratio = [1-sizes["All"][i] / sizes["None"][i] for i in range(len(tests))]
print(min(ratio), max(ratio))
print(np.mean(ratio))

In [ ]:
ratio = [1-times["All"][i] / times["None"][i] for i in range(len(tests))]
print(min(ratio), max(ratio))
print(np.mean(ratio))

Comparing all optimisations vs none:  
The difference in filesize varies from -14% to -39%, with an average of -23%.  
Execution time at best decreases by 90%, but in the worst example only increases by 2.2%, and it decreases in the majority of cases, particularly the more functional style programs.  
TODO: Are such figures meaningful for hand picked examples? Particularly the average.

## Specific IR optimisations (inlining, tail calls, uncurrying)
I now look at the effect of removing one of inlining, tail calls, and uncurrying on the resulting program to assess their relative impact. Copy propagation/folding and dead code elimination is kept in all cases.  

One additional factor not shown is that tail call optimisation can prevent a program from exceeding the maximum call stack size by avoiding nested function calls. Here is a program that illustrates this behaviour:
```
let rec f x =
  if x = 0 then 0 
  else f (x-1)

let a = f 100000
```
Without tail call optimisations, each recursive call adds another stack frame, eventually exhausting the available stack space and causing an error.

In [ ]:
data = {}
labels = ["All", "-inline", "-uncurry", "-tail-call"]
# Check file to see formatting
with open("../../wasm-of-ocaml/benchmarks/evaluation/specific_opts.txt") as f:
    for i in labels:
        f.readline()
        line = f.readline().strip().split()
        while line != []:
            if line[0] not in data:
                data[line[0]] = {}
            data[line[0]][i] = \
              {"time" : float(line[1]), "error" : float(line[2]), "heap": float(line[3]), "filesize" : float(line[4])}
            line = f.readline().strip().split()
tests = data.keys()

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(12,10))
ax = axs[0]
times = {i: [data[test][i]["time"] for test in tests] for i in labels}
errors = {i: [data[test][i]["error"] for test in tests] for i in labels}
times, errors = normalise(times, errors, "All", True)
#times = {lang : [means[lang][b] for b in benchmarks if b != "arith"] for lang in means if lang != "Grain"}
bar_plot(ax, times, errors=errors)
ax.set_title("Execution time")
ax.set_xticks([])
ax.set_ylabel("relative time")

ax = axs[1]
mem = {i: [data[test][i]["heap"] for test in tests] for i in labels}
mem, _ = normalise(mem, None, "All", True)
handlers, labels = bar_plot(ax, mem, legend=True)
ax.set_title("Heap usage")
ax.set_xticks([])
ax.set_yscale("log")
ax.set_ylabel("Memory (bytes)")

ax = axs[2]
sizes = {i: [data[test][i]["filesize"] for test in tests] for i in labels}
sizes, _ = normalise(sizes, None, "All", True)
handlers, labels = bar_plot(ax, sizes, legend=True)
ax.set_title("Filesize")
ax.set_xticks(range(len(tests)))
ax.set_xticklabels(tests, rotation=90)
ax.set_ylabel("size (bytes)")
ax.set_ylim((0.8,1.3))

fig.legend(handlers, labels, loc='center left')
plt.subplots_adjust(left=0.15)

Is this a slightly confusing way to display this information? i.e. worse score when removed => better performance when present?
Would be better to just show effect of each optimisation in isolation?

From the timing graph we see two expected results: In functions where currying can be removed, not creating additional closures can significantly improve performance, running up to twice as fast.  Also, tail call optimisation generally reduces performance due to the overhead of implementing it, the benefit being that programs will not give an error due to using all available stack space in tail recursive calls.  
In terms of memory consumption, as expected there is a significant improvement by replacing curried calls with passing the arguments as a tuple where possible. In the case of the arithmetic programs, these closures make up almost all of the memory allocations performed by the program, so removing them can reduce memory usage almost entirely.  
Lastly, we see that removing currying also reduces the size of programs, as extra functions to create closures are left out of the code. By limiting the amount of extra code that can be introduced by inlining, the code bloat it causes is limited, and in some cases programs even become smaller due to a function being completely inlined and removed.

## Phase order and number of iterations

In [ ]:
data = {}
labels = ["0", "1", "2", "3", "4", "5"]
# Check file to see formatting
with open("../../wasm-of-ocaml/benchmarks/evaluation/iterations.txt") as f:
    for i in labels:
        f.readline()
        line = f.readline().strip().split()
        while line != []:
            if line[0] not in data:
                data[line[0]] = {}
            data[line[0]][i] = \
              {"time" : float(line[1]), "error" : float(line[2]), "heap": float(line[3]), "filesize" : float(line[4])}
            line = f.readline().strip().split()
tests = data.keys()

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(12,10))
ax = axs[0]
times = {i: [data[test][i]["time"] for test in tests] for i in labels}
errors = {i: [data[test][i]["error"] for test in tests] for i in labels}
times, errors = normalise(times, errors, "0", True)
#times = {lang : [means[lang][b] for b in benchmarks if b != "arith"] for lang in means if lang != "Grain"}
bar_plot(ax, times, errors=errors)
ax.set_title("Execution time")
ax.set_xticks([])
ax.set_ylabel("relative time")

ax = axs[1]
mem = {i: [data[test][i]["heap"] for test in tests] for i in labels}
mem, _ = normalise(mem, None, "0", True)
handlers, labels = bar_plot(ax, mem, legend=True)
ax.set_title("Heap usage")
ax.set_xticks([])
ax.set_yscale("log")
ax.set_ylabel("Memory (bytes)")

ax = axs[2]
sizes = {i: [data[test][i]["filesize"] for test in tests] for i in labels}
sizes, _ = normalise(sizes, None, "0", True)
handlers, labels = bar_plot(ax, sizes, legend=True)
ax.set_title("Filesize")
ax.set_xticks(range(len(tests)))
ax.set_xticklabels(tests, rotation=90)
ax.set_ylabel("size (bytes)");

fig.legend(handlers, labels, loc='center left');
plt.subplots_adjust(left=0.15)

From these plots we see that performance rarely gets worse with more iterations, and that further improvement after three iterations only happens in a couple instances. Where there are improvements from performing more than three iterations, these tend to be much smaller than the improvements already achieved, hence 3 iterations is chosen as the number of times to run the set of optimisation passes for all other analysis.  

Performing multiple passes of the iterations reduces the significance of the ordering of passes. Still, the more complex optimisations such as inlining or uncurrying functions tend to rely on some values being propagated through the code initially produced. As such, these information propagation passes such as CSE and constant porpagation are run first.  
Functions are then inlined where beneficial, and those functions which are fully applied but are not selected to be inlined are then uncurried. Lastly, dead assignment elimination removes any now useless definitions and tail call optimisation attempts to optimise suitable recursive functions that have not been removed.  

Inlining in particular reveals new opportunities to propagate values, hence the benefit of running iterations in multiple passes.

In [ ]:
data = {}
labels = ["1", "2", "3", "4", "5"]
# Check file to see formatting
with open("../../wasm-of-ocaml/benchmarks/evaluation/phaseorder.txt") as f:
    for i in labels:
        f.readline()
        line = f.readline().strip().split()
        while line != []:
            if line[0] not in data:
                data[line[0]] = {}
            data[line[0]][i] = \
              {"time" : float(line[1]), "error" : float(line[2]), "heap": float(line[3]), "filesize" : float(line[4])}
            line = f.readline().strip().split()
tests = data.keys()

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(12,10))
ax = axs[0]
times = {i: [data[test][i]["time"] for test in tests] for i in labels}
errors = {i: [data[test][i]["error"] for test in tests] for i in labels}
times, errors = normalise(times, errors, "1", True)
#times = {lang : [means[lang][b] for b in benchmarks if b != "arith"] for lang in means if lang != "Grain"}
bar_plot(ax, times, errors=errors)
ax.set_title("Execution time")
ax.set_xticks([])
ax.set_ylabel("relative time")

ax = axs[1]
mem = {i: [data[test][i]["heap"] for test in tests] for i in labels}
mem, _ = normalise(mem, None, "1", True)
handlers, labels = bar_plot(ax, mem, legend=True)
ax.set_title("Heap usage")
ax.set_xticks([])
ax.set_ylabel("Memory (bytes)")

ax = axs[2]
sizes = {i: [data[test][i]["filesize"] for test in tests] for i in labels}
sizes, _ = normalise(sizes, None, "1", True)
handlers, labels = bar_plot(ax, sizes, legend=True)
ax.set_title("Filesize")
ax.set_xticks(range(len(tests)))
ax.set_xticklabels(tests, rotation=90)
ax.set_ylabel("size (bytes)");

fig.legend(handlers, labels, loc='center left');
plt.subplots_adjust(left=0.15)

Have swapped best choice i.e. now doing uncurrying before inlining and dead code last. (but only a relative graph so doesn't matter if its consistent or not)
Various orderings of the IR optimisations were considered, treating all of the optimisations which just propagate values as one group i.e. copy/constant propagation and CSE. 5 orderings of propagation, uncurrying, inlining, tail calls, and dead code elimination are shown above. In most instances, the choice of ordering had no impact on heap usage or filesize so this is not shown.  
From the timing plots, options 4 and 5 are significantly worse in most cases. Between the other three, there is no clearly better choice as it appears to depend on the program compiled. Therefore, I use ordering 1 in all other tests:  
propagate-uncurry-inline-tailcall-dead code  
Intuitively, propagating known values and evaluating constant expressions makes sense to do first as this exposes opportunities to perform other optimisation. Uncurrying is done next as it potentially removes several intermediate functions, followed by inlining and tailcall optimisation. The order of these two makes no difference, as we do not inline recursive functions whereas we only tailcall optimise recursive functions. Finally, after all of these changes have been made, dead code elimination removes all values which are no longer required.  
The fact that this is repeated 3 times reduces how sensitive the output is to the ordering chosen.


The 5 examples are shown just for an idea of the variation caused by the ordering, as even with inlining and tailcall optimisation being interchangeable there are 96 distinct orderings, were they to be considered exhaustively.

## Outputting data for MATLAB

In [ ]:
labels = ["0", "1", "2", "3", "4", "5"]
cases = [0, 3, 5, 6, 7, 9]
print([list(tests)[i] for i in cases])
print("[" +  "; ".join([" ".join([str(times[label][i]) for label in labels]) for i in cases]) + "]")
print("[" +  "; ".join([" ".join([str(errors[label][i]) for label in labels]) for i in cases]) + "]")
print()
print("[" +  "; ".join([" ".join([str(mem[label][i]) for label in labels]) for i in cases]) + "]")
print()
print("[" +  "; ".join([" ".join([str(sizes[label][i]) for label in labels]) for i in cases]) + "]")

In [ ]:
times["-inline"]